In [ ]:
import numpy as np
import math
#Trabalho desenvolvido por Lucas Rufino e Ronald Albert

In [ ]:
class Tabuleiro:
    #Função para criação do tabuleiro, se nada for passado ao parâmetro 'queens'
    #ele irá usar a função 'gerarTabuleiro()' e gerar um tabuleiro aleatório para o
    #self.queens
    def __init__(self, n, queens=[], binString=''):
        self.n = n
        if(not np.array_equal(queens, []) and binString == ''):
            self.queens = queens
            self.binString = self.representacaoBinaria(1, self.n)
        elif (np.array_equal(queens, []) and binString != ''):
            self.binString = binString
            self.queens = self.gerarTabuleiroDeBin(self.binString)
        elif (not np.array_equal(queens, []) and binString != ''):
            self.queens = queens
            self.binString = binString
        else:
            self.queens = self.gerarTabuleiro()
            self.binString = self.representacaoBinaria(1, self.n)
                
    def gerarTabuleiro(self):
        queens = np.random.randint(1, self.n + 1, size=(self.n))
        return queens
    
    def gerarTabuleiroDeBin(self, queens):
        nBits = math.ceil(math.log(self.n, 2))
        tabuleiro = [(int(queens[i:i+nBits], 2) + 1) for i in range(0, len(queens), nBits)]
        return tabuleiro
    
    def representacaoBinaria(self, minValue, maxValue):
        binString = ''
        nBits = math.ceil(math.log(self.n, 2))
        for e in self.queens:
            binString += bin(e - minValue)[2:].zfill(nBits)
        return binString
    
    #Função de avaliação do tabuleiro, itera por todas as colunas
    #e para cada iteração de uma coluna i, a função avalia a posição
    #da rainha nessa coluna iterada, e checa nas colunas seguintes a i
    # (>= i + 1), quantas rainhas podem ser atacadas pela rainha da coluna i.
    def avaliarTabuleiro(self):
        if hasattr(self, 'n_ataques'):
            return self.n_ataques
        
        n_ataques = 0
        for i in range(0, self.n):
            for j in range(i + 1, self.n):
                if self.queens[i] == self.queens[j]:
                    n_ataques += 1
                elif self.queens[i] + (j - i) == self.queens[j]:
                    n_ataques += 1
                elif self.queens[i] - (j - i) == self.queens[j]:
                    n_ataques += 1
        self.n_ataques = n_ataques
        return n_ataques
    
    #Função para printar o tabuleiro:
    def print_tabuleiro(self):
        matriz_rainha = np.zeros((self.n,self.n))
        for i in range(self.n):
            for j in range(self.n):
                matriz_rainha[i][j] = 0
        for i in range(self.n):            
            for j in range(self.n):
                if self.queens[j] == i+1:                
                    matriz_rainha[i][j] = 1 
        matriz_texto = "+"
        for i in range(self.n):
            matriz_texto += "---"
        matriz_texto += "+\n|"
        for i in range(self.n):
            matriz_texto += ""
            for j in range(self.n):
                if matriz_rainha[i][j] == 0:
                    matriz_texto += " . "
                else:
                    matriz_texto += " Q "
                if j == self.n-1:
                    if i == self.n-1:
                        matriz_texto += "|\n"
                    else:
                        matriz_texto += "|\n|"
        matriz_texto += "+"
        for i in range(self.n):
            matriz_texto += "---"
        matriz_texto += "+\n"
        matriz_texto += "|"
        for i in range(self.n):
            matriz_texto += "   "
        matriz_texto += "|\n"
        matriz_texto += "+"
        for i in range(self.n):
            matriz_texto += "---"
        matriz_texto += "+\n"
        print(matriz_texto)
        
a = Tabuleiro(8, binString='100011110100110000101100')
print(a.queens)
print(a.binString)

In [ ]:
class Populacao:
    def __init__(self, n, size, pc, pm, tabuleiros = []):
        self.n = n
        self.size = size
        self.pc = pc
        self.pm = pm
        self.tabuleiros = tabuleiros if not np.array_equal(tabuleiros, []) else self.gerarPopulacao()
        self.probabilidades = self.construirRoletaViciada()
        self.popIntermediaria = self.construirPopulacaoIntermediaria()
        self.popIntermediaria = self.realizarCrossover()
        self.popIntermediaria = self.realizarMutacao()
    
    def gerarPopulacao(self):
        tabuleiros = []
        for i in range(0, self.n):
            tabuleiros.append(list(Tabuleiro(self.size).queens))
        return tabuleiros
    
    def construirRoletaViciada(self):
        probabilidades = []
        totalSum = 0
        for i in self.tabuleiros:
            totalSum += Tabuleiro(self.size, queens=i).avaliarTabuleiro()
            probabilidades.append(totalSum)
        
        probabilidades = list(map(lambda x: x/totalSum, probabilidades))
        self.probabilidades = probabilidades
        return probabilidades
    
    def construirPopulacaoIntermediaria(self):
        tabuleiros = []
        for i in range(0, self.n):
            randomNumber = np.random.random()
            for i in range(0, len(self.probabilidades)):
                if(self.probabilidades[i] > randomNumber):
                    tabuleiros.append(self.tabuleiros[i])
                    break
        self.populacao_intermediaria = tabuleiros
        return tabuleiros
    
    def realizarCrossover(self):
        tabuleiros = []
        for i in range(0, self.n, 2):
            randomNumber = np.random.random()
            if(randomNumber < self.pc):
                randomIndex = np.random.randint(1, self.size)
                novoTabuleiro1 = self.popIntermediaria[i][:(randomIndex)] + self.popIntermediaria[i + 1][(randomIndex):]
                novoTabuleiro2 = self.popIntermediaria[i + 1][:(randomIndex)] + self.popIntermediaria[i][(randomIndex):]
                tabuleiros.append(novoTabuleiro1)
                tabuleiros.append(novoTabuleiro2)
            else:
                tabuleiros.append(self.popIntermediaria[i])
                tabuleiros.append(self.popIntermediaria[i + 1])
        
        return tabuleiros
    
    def realizarMutacao(self):
        tabuleiros = []
        for i in self.popIntermediaria:
            randomNumber = np.random.random()
            novoTabuleiro = i
            if(randomNumber < self.pm):
                randomIndex = np.random.randint(0, self.size)
                novoTabuleiro[randomIndex] = np.random.randint(1, self.size + 1)
            tabuleiros.append(novoTabuleiro)
        
        return tabuleiros
        

In [ ]:
class PopulacaoBinario:
    def __init__(self, n, size, pc, pm, tabuleiros = []):
        self.n = n
        self.size = size
        self.pc = pc
        self.pm = pm
        self.tabuleiros = tabuleiros if not np.array_equal(tabuleiros, []) else self.gerarPopulacao()
        self.probabilidades = self.construirRoletaViciada()
        self.popIntermediaria = self.construirPopulacaoIntermediaria()
        self.popIntermediaria = self.realizarCrossover()
        self.popIntermediaria = self.realizarMutacao()
    
    def gerarPopulacao(self):
        tabuleiros = []
        for i in range(0, self.n):
            tabuleiros.append(Tabuleiro(self.size).binString)
        
        return tabuleiros
    
    def construirRoletaViciada(self):
        probabilidades = []
        totalSum = 0
        for i in self.tabuleiros:
            totalSum += Tabuleiro(self.size, binString=i).avaliarTabuleiro()
            probabilidades.append(totalSum)
        
        probabilidades = list(map(lambda x: x/totalSum, probabilidades))
        self.probabilidades = probabilidades
        return probabilidades
    
    def construirPopulacaoIntermediaria(self):
        tabuleiros = []
        for i in range(0, self.n):
            randomNumber = np.random.random()
            for i in range(0, len(self.probabilidades)):
                if(self.probabilidades[i] > randomNumber):
                    tabuleiros.append(self.tabuleiros[i])
                    break
        self.populacao_intermediaria = tabuleiros
        return tabuleiros
    
    def realizarCrossover(self):
        tabuleiros = []
        nBits = math.ceil(math.log(self.size, 2))
        for i in range(0, self.n, 2):
            randomNumber = np.random.random()
            if(randomNumber < self.pc):
                randomIndex = np.random.randint(1, self.size)
                novaStringBin1 = self.popIntermediaria[i][:(randomIndex * nBits)] + self.popIntermediaria[i + 1][(randomIndex * nBits):]
                novaStringBin2 = self.popIntermediaria[i + 1][:(randomIndex * nBits)] + self.popIntermediaria[i][(randomIndex * nBits):]
                tabuleiros.append(novaStringBin1)
                tabuleiros.append(novaStringBin2)
            else:
                tabuleiros.append(self.popIntermediaria[i])
                tabuleiros.append(self.popIntermediaria[i + 1])
        
        return tabuleiros
    
    def realizarMutacao(self):
        tabuleiros = []
        nBits = math.ceil(math.log(self.size, 2))
        for i in self.popIntermediaria:
            randomNumber = np.random.random()
            novaBinString = i
            if(randomNumber < self.pm):
                randomIndex = np.random.randint(0, nBits * self.size)
                listBinString = list(novaBinString)
                listBinString[randomIndex] =  '1' if listBinString[randomIndex] == '0' else '0'
                novaBinString = "".join(listBinString)
            tabuleiros.append(novaBinString)
        
        return tabuleiros

In [ ]:
def algoritmoGeneticoBinario(n_it, tamanho_pop, tamanho_tab, p_cross, p_mut, elitismo=False):
    popCorrente = PopulacaoBinario(tamanho_pop, tamanho_tab, p_cross, p_mut)
    for i in range(0, n_it):
        popCorrente = PopulacaoBinario(tamanho_pop, tamanho_tab, p_cross, p_mut, popCorrente.popIntermediaria)
    
    return popCorrente

In [ ]:
def algoritmoGenetico(n_it, tamanho_pop, tamanho_tab, p_cross, p_mut, elitismo=False):
    popCorrente = Populacao(tamanho_pop, tamanho_tab, p_cross, p_mut)
    for i in range(0, n_it):
        popCorrente = Populacao(tamanho_pop, tamanho_tab, p_cross, p_mut, popCorrente.popIntermediaria)
    
    return popCorrente